### 0. 패키지 불러오기

In [14]:
import torch
import pandas as pd
import numpy as np
import time
import datetime
import gc

from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel
from transformers import BertModel, DistilBertModel
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import DistilBertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import DistilBertModel, DistilBertConfig
from transformers import get_linear_schedule_with_warmup

## warning 제거
import warnings
warnings.filterwarnings("ignore")

In [15]:
# GPU 여부 확인
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [16]:
# 디바이스 GPU로 세팅하기 (가능하지 않을 시, CPU로 변경)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [17]:
torch.cuda.is_available()

True

### 1. 데이터 불러오기

In [18]:
df= pd.read_excel("C:/Users/User/Desktop/논문준비/news/뉴스 차종 라벨링_스택_v3.xlsx",names=['date', 'sentence','source', 'label','pos_neg', 'contents'])

In [19]:
# 문장 숫자 출력
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 1,799



In [20]:
# 5개 샘플 로우 출력
df.sample(5)

,date,sentence,source,label,pos_neg,contents
976,2019.03.26.,벤츠 주요 차종 자차보험료 인하효과 누려,교통신문,2,1,메르세데스-벤츠코리아가 최근 보험개발원에서 실시한 차량 모델 등급 평가 결과 C클래...
736,2020.01.23.,"현대차, '제이디파워' 인도 신차품질조사서 5개 차종 1위 선정",매일일보,1,1,"D Power)가 발표한 '2019 인도 신차품질조사((IQS, Initial Qu..."
1573,2019.05.23.,'화재 위험' 포르쉐 파나메라 2800여대 리콜,경향신문,2,2,"화재 위험이 발견된 포르쉐 파나메라를 비롯해 혼다, 푸조 등 수입차 12개 차종, ..."
38,2020.01.30.,"포스코케미칼, 전기차 시장 확대로 배터리소재 증설-미래에셋",철강금속신문,6,0,미래에셋대우는 30일 포스코케미칼에 대해 전기차 시장 확대에 따른 배터리소재 증설이...
2,2020.03.30.,"[차종은 쇼] 40,50, 60 대 분들에게 추천하는 성형 vs 시술은? (이태근 ...",의학채널 비온뒤,0,0,오늘 차종은PD가 진행하는 차종은쇼에서 오스킨 성형외과 이태근 원장과 연세고운피부과...


In [21]:
dt=pd.DataFrame()

In [22]:
# 데이터 sentence, label 구분하기
sentences = (df['sentence'].astype(str)+df['contents'].astype(str)).values
#labels = df.label.values
labels = df.pos_neg.values
#pos_neg= df.pos_neg.values

In [23]:
## 문장 확인
sentences

array(['힘 센 놈, 날렵한 놈, 신기한 놈…베일 벗는 차차차대표하는 차종들이 새 얼굴로 바뀌었다. 하반기에도 신차 여러 종이 출시를 준비 중이다. 8세대 신형... 3세대 K5, 쏘나타 질주 따라잡을까 최근 10여년 동안 거셌던 스포츠실용차(SUV) 바람에 가장 큰 영향을 받은 차종...',
       '[치과 라이브] 사랑니 수술 발치(뉴연세치과 류성용 원장 & 차종은 PD)사랑니는 꼭 발치해야 할까요? 사람마다 달라서 발치해야 하는 경우와 아닌 경우가 나눠지게 됩니다. 오늘 뉴연세치과 류성용 원장과 차종은 PD가 그 과정과 결과를 자세히 알려드립니다! #사랑니 #치과 #류성용원장',
       '[차종은 쇼] 40,50, 60 대 분들에게 추천하는 성형 vs 시술은? (이태근 성형외...오늘 차종은PD가 진행하는 차종은쇼에서 오스킨 성형외과 이태근 원장과 연세고운피부과 윤나영 원장을 모시고 그 해결책을 알려드립니다. 시술과 성형 중 나에게 맞는 초이스가 무엇일지 궁금하시다면 오늘...',
       ...,
       '쌍용차·BMW·닛산 등 28개차종 3만3천대 리콜…KATO 기중기 254대도국토교통부는 쌍용차·BMW·닛산 등 28개 차종 3만2천951대와 2013년 3월26일 이후 국내에서 판매된 KATO사의... BMW X6 xDrive30d 등 7개 차종 8천680대는 카시트 부착 장치의 용접 불량으로 결합부가 이탈돼 사고 발생 시 제...',
       '5월 4주 주요 제조업 전망2만 대(전년 동월 대비 –13%) □ 현대·기아차, BMW, 벤츠 등 126개 차종 55만 대 리콜 - 국토교통부, 지난 22일 "7개사 차종 약 55만 대에서 결함 발견돼 리콜" - 리콜 대상 : 현대차, 기아차, BMW코리아, 메르세데스...',
       "코로나19 후폭풍으로 중고차 시장 ‘꽁꽁’최근 인기가 높은 SUV 차종은 견조한 시세를 유지했다. 그 중에서도 투싼과 쏘렌토의 약진이 두드러졌다. '올... 지난달에도 SUV 차종이 상대적으로

In [24]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel
from transformers import BertModel, DistilBertModel
#from tokenization_kobert import KoBertTokenizer
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import DistilBertTokenizer

tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

# Load the BERT tokenizer.
#print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

#model = BertModel.from_pretrained('monologg/kobert')
#tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
#tokenizer = AutoTokenizer.from_pretrained(path)

## Base Model (108M)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")

# Large Model (334M)
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")


#bert_model = BertModel.from_pretrained('monologg/kobert')

In [25]:
# Print the original sentence.
print(' Original: ', sentences[0])

print("")

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

print("")

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  힘 센 놈, 날렵한 놈, 신기한 놈…베일 벗는 차차차대표하는 차종들이 새 얼굴로 바뀌었다. 하반기에도 신차 여러 종이 출시를 준비 중이다. 8세대 신형... 3세대 K5, 쏘나타 질주 따라잡을까 최근 10여년 동안 거셌던 스포츠실용차(SUV) 바람에 가장 큰 영향을 받은 차종...

Tokenized:  ['힘', '센', '놈', ',', '날', '##렵', '##한', '놈', ',', '신기', '##한', '놈', '[UNK]', '베', '##일', '벗', '##는', '차', '##차', '##차', '##대표', '##하는', '차', '##종', '##들이', '새', '얼굴', '##로', '바뀌', '##었다', '.', '하', '##반', '##기에', '##도', '신', '##차', '여러', '종이', '출시', '##를', '준비', '중이다', '.', '8', '##세대', '신', '##형', '.', '.', '.', '3', '##세대', 'K', '##5', ',', '쏘', '##나', '##타', '질', '##주', '따라', '##잡을', '##까', '최근', '10', '##여', '##년', '동안', '거', '##셌', '##던', '스포츠', '##실', '##용', '##차', '(', 'S', '##U', '##V', ')', '바람에', '가장', '큰', '영향을', '받은', '차', '##종', '.', '.', '.']

Token IDs:  [3538, 1866, 692, 15, 595, 4121, 4047, 692, 15, 11302, 4047, 692, 1, 1543, 4046, 1537, 4008, 2839, 4495, 4495, 8610, 7966, 2839, 4197, 7967, 1807, 8488, 4091, 9239, 8217, 17, 3352, 4593, 8319, 4029, 2005, 4495, 8741, 18317, 22693, 4180, 9242

In [26]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  129


In [27]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Original:  힘 센 놈, 날렵한 놈, 신기한 놈…베일 벗는 차차차대표하는 차종들이 새 얼굴로 바뀌었다. 하반기에도 신차 여러 종이 출시를 준비 중이다. 8세대 신형... 3세대 K5, 쏘나타 질주 따라잡을까 최근 10여년 동안 거셌던 스포츠실용차(SUV) 바람에 가장 큰 영향을 받은 차종...
Token IDs: tensor([    2,  3538,  1866,   692,    15,   595,  4121,  4047,   692,    15,
        11302,  4047,   692,     1,  1543,  4046,  1537,  4008,  2839,  4495,
         4495,  8610,  7966,  2839,  4197,  7967,  1807,  8488,  4091,  9239,
         8217,    17,  3352,  4593,  8319,  4029,  2005,  4495,  8741, 18317,
        22693,  4180,  9242, 27613,    17,    27, 11372,  2005,  4572,    17,
           17,    17,    22, 11372,    44,  4044,    15,  2088,  4136,  4361,
         2694,  4043,  8738, 19816,  4109, 12219,  8240,  4327,  4482, 10921,
          248,  6711,  4217, 18000,  4353,  4429,  4495,    11,    52,  5167,
         4604,    12, 20473,  8547,  3089, 17283,  9838,  2839,  4197,    17,
           17,    17,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     

In [28]:
from torch.utils.data import TensorDataset, random_split
import torch

torch.manual_seed(42)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,619 training samples
  180 validation samples


In [29]:
train_dataset

In [30]:
val_dataset

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [51]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import DistilBertModel, DistilBertConfig
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

i=7
path= 'C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/serialization_dir/my_first_training'+str(i)+'/'


#config = DistilBertConfig.from_pretrained('C:/Users/User/Desktop/논문준비/news/DistilKoBERT-master/distillation/training_configs/distilbert-base-cased/distilkobert_6_layer.json', output_hidden_states=True)
#model = BertForSequenceClassification.from_pretrained(
#    'beomi/kcbert-base', # Use the 12-layer BERT model, with an uncased vocab.
#    num_labels = len(df['label'].unique()), # The number of output labels--2 for binary classification.
#                    # You can increase this for multi-class tasks.   
#    output_attentions = False, # Whether the model returns attentions weights.
#    output_hidden_states = False # Whether the model returns all hidden-states.
#)

## 사용 가능한 pre-trained 모델들
##monologg/koelectra-small-discriminator
#monologg/distilkobert
#monologg/koelectra-base-discriminator
#monologg/kobert
#beomi/kcbert-base
#albert-base-v2

model = DistilBertForSequenceClassification.from_pretrained(
    path, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(df['label'].unique()),
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

model.cuda()

Some weights of the model checkpoint at C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/serialization_dir/my_first_training7/ were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at C:/Users/User/Desktop/논문준비/n

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [52]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 104 different named parameters.

==== Embedding Layer ====

distilbert.embeddings.word_embeddings.weight            (30000, 768)
distilbert.embeddings.position_embeddings.weight          (300, 768)
distilbert.embeddings.LayerNorm.weight                        (768,)
distilbert.embeddings.LayerNorm.bias                          (768,)
distilbert.transformer.layer.0.attention.q_lin.weight     (768, 768)

==== First Transformer ====

distilbert.transformer.layer.0.attention.q_lin.bias           (768,)
distilbert.transformer.layer.0.attention.k_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.k_lin.bias           (768,)
distilbert.transformer.layer.0.attention.v_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.v_lin.bias           (768,)
distilbert.transformer.layer.0.attention.out_lin.weight   (768, 768)
distilbert.transformer.layer.0.attention.out_lin.bias         (768,)
distilbert.transformer.layer.0.sa_layer_norm.weight           (

In [53]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [54]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 30

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [55]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [56]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [57]:
import gc
gc.collect()

732

In [58]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# iteration setting
l=0

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
        #                     token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    
    
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   #token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
        ## Confusion Matrix 생성
        if l==0:
            pred_result= pd.DataFrame(np.argmax(logits, axis=1).flatten()).rename({0:'pred_result'},axis=1)
            label_result= pd.DataFrame(label_ids.flatten()).rename({0:'label_result'},axis=1)
            fnl_dt= pd.concat([pred_result,label_result],axis=1)
            fnl_dt['epochs']= epoch_i
            l= l+1
            
        else:
            pred_result= pd.DataFrame(np.argmax(logits, axis=1).flatten()).rename({0:'pred_result'},axis=1)
            label_result= pd.DataFrame(label_ids.flatten()).rename({0:'label_result'},axis=1)
            result= pd.concat([pred_result,label_result],axis=1)
            result['epochs']= epoch_i
            fnl_dt= pd.concat([fnl_dt,result])
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 30 ========
Training...
  Batch    40  of     51.    Elapsed: 0:00:05.

  Average training loss: 1.135
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.603
  Validation Loss: 0.861
  Validation took: 0:00:00

======== Epoch 2 / 30 ========
Training...
  Batch    40  of     51.    Elapsed: 0:00:05.

  Average training loss: 0.930
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.518
  Validation Loss: 0.847
  Validation took: 0:00:00

======== Epoch 3 / 30 ========
Training...
  Batch    40  of     51.    Elapsed: 0:00:05.

  Average training loss: 0.915
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.629
  Validation Loss: 0.776
  Validation took: 0:00:00

======== Epoch 4 / 30 ========
Training...
  Batch    40  of     51.    Elapsed: 0:00:05.

  Average training loss: 0.841
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.678
  Validation Loss: 0.757
  Validation took: 0:00:00

======== Ep

### distil 모델 기능에 대한 데이터 생성

dt=pd.DataFrame()

total_sec= time.time()-total_t0

test_dt= pd.DataFrame(training_stats)

test_dt['total_seconds']=total_sec
test_dt['model_index']=i

dt= pd.concat([test_dt,dt])

dt

In [ ]:
#dt.to_csv("C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/test.txt")

### Confusion Matrix 생성

In [59]:
fnl_dt.to_csv('confusion_matrix_distilkcbert_even.txt')

In [42]:
fnl_dt

,pred_result,label_result,epochs
0,1,1,0
1,1,2,0
2,1,1,0
3,1,1,0
4,2,2,0
5,0,0,0
6,1,1,0
7,1,1,0
8,0,0,0
9,1,1,0
